In [ ]:
!pip install scikit-learn-extra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.23.5
!pip install --upgrade scikit-learn-extra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is i

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
^C


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score, mutual_info_score, accuracy_score, normalized_mutual_info_score
from scipy.spatial.distance import pdist, squareform, cdist
import os
import kagglehub
# from sklearn.feature_selection import mutual_info_classif
from sklearn_extra.cluster import KMedoids

In [ ]:
path_iris = kagglehub.dataset_download("himanshunakrani/iris-dataset")
f_path_iris = os.path.join(path_iris, 'iris.csv')
iris_df = pd.read_csv(f_path_iris)
# Extract labels (species)
iris_labels = iris_df['species']
# Remove label from the features
iris_features = iris_df.drop(columns=['species'])
# Convert to numpy
iris_data = iris_features.values


# Preprocessing for AI Global Index Dataset
def preprocess_ai_index(data):
    # Separate numerical and categorical columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns

    scaler = MinMaxScaler()
    num_scaled = scaler.fit_transform(data[numeric_cols]) if len(numeric_cols) > 0 else np.array([])

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cat_encoded = encoder.fit_transform(data[categorical_cols]) if len(categorical_cols) > 0 else np.array([])

    if num_scaled.size and cat_encoded.size:
        return np.hstack((num_scaled, cat_encoded))
    elif num_scaled.size:
        return num_scaled
    else:
        return cat_encoded

path_ai_index = kagglehub.dataset_download("katerynameleshenko/ai-index")
f_path_ai_index = os.path.join(path_ai_index, 'AI_index_db.csv')
ai_df = pd.read_csv(f_path_ai_index)
ai_df = ai_df.dropna()
# Extract the 'Cluster' column as the label
ai_labels = ai_df['Cluster']
# Remove the 'Cluster' column from the features
ai_df_features = ai_df.drop(columns=['Cluster'])
# Preprocess the remaining features
ai_data = preprocess_ai_index(ai_df_features)

path_earthquakes= kagglehub.dataset_download("shreyasur965/recent-earthquakes")
f_path_earthquakes = os.path.join(path_earthquakes, 'earthquakes.csv')
earthquake_df = pd.read_csv(f_path_earthquakes)
earthquake_df = earthquake_df[['magnitude', 'felt', 'cdi','mmi','tsunami','sig','depth', 'latitude', 'longitude', 'alert']].dropna()
# Extract the alert labels
earthquake_data_alerts = earthquake_df['alert']
alert_encoded = LabelEncoder().fit_transform(earthquake_data_alerts)
# Remove the label from features
earthquake_data_features = earthquake_df.drop(columns=['alert'])
# Scale the numeric features
earthquake_data = StandardScaler().fit_transform(earthquake_data_features)

100%|██████████| 0.98k/0.98k [00:00<00:00, 1.92MB/s]

Extracting files...


100%|██████████| 2.38k/2.38k [00:00<00:00, 1.93MB/s]

Extracting files...


100%|██████████| 214k/214k [00:00<00:00, 64.6MB/s]

Extracting files...


In [ ]:
class BASE_KMEDOIDS:
    def __init__(self, k, max_iters=100):
        self.k = k
        self.max_iters = max_iters

    def fit(self, data):
        self.data = data
        self.is_dataframe = isinstance(data, pd.DataFrame)

        if self.is_dataframe:
            data_np = data.to_numpy()
        else:
            data_np = data

        n_samples, n_features = data_np.shape
        medoid_indices = np.random.choice(n_samples, self.k, replace=False)

        # Handle medoid initialization correctly
        self.medoids = data.iloc[medoid_indices].to_numpy() if self.is_dataframe else data[medoid_indices]
        self.labels = np.zeros(n_samples, dtype=int)

        for _ in range(self.max_iters):
            prev_labels = self.labels.copy()

            # Compute Manhattan distances
            distances = np.sum(np.abs(data_np[:, np.newaxis] - self.medoids), axis=2)
            self.labels = np.argmin(distances, axis=1)

            # Update medoids
            for i in range(self.k):
                cluster_indices = np.where(self.labels == i)[0]
                if len(cluster_indices) == 0:
                    continue

                cluster_points = data_np[cluster_indices]
                min_cost = float('inf')
                best_medoid = None

                for idx in cluster_indices:
                    cost = np.sum(np.sum(np.abs(data_np[idx] - cluster_points), axis=1))
                    if cost < min_cost:
                        min_cost = cost
                        best_medoid = data_np[idx]

                self.medoids[i] = best_medoid

            # Convergence check
            if np.array_equal(self.labels, prev_labels):
                break

        return self.labels, self.medoids

    def _to_numpy(self, d):
      if self.is_dataframe:
          # Only call .values if d is a DataFrame or Series
          return d.values if hasattr(d, 'values') else np.array(d)
      return d

    def _get_row(self, idx):
        return self.data.iloc[idx].values if self.is_dataframe else self.data[idx]

    def _get_rows(self, indices):
        return self._to_numpy(self.data.iloc[indices]) if self.is_dataframe else self.data[indices]


In [ ]:
def sklearn_kmedoids(features, n_clusters):
    model = KMedoids(n_clusters=n_clusters, random_state=42)
    return model.fit_predict(features)

In [ ]:
def compute_diameter(X, labels):
    """
    Computes the average 'diameter' across all clusters,
    where 'diameter' of a cluster is the maximum distance
    between any two points in that cluster.
    """
    unique_labels = np.unique(labels)
    diameters = []
    for lbl in unique_labels:
        cluster_points = X[labels == lbl]
        if len(cluster_points) > 1:
            # pairwise distances in the cluster
            dist_matrix = squareform(pdist(cluster_points))
            diameters.append(dist_matrix.max())
        else:
            # A single point has diameter 0
            diameters.append(0.0)
    return np.mean(diameters)


def compute_split(X, labels):
    """
    An example 'split' metric: ratio of the size of the largest cluster
    to the size of the smallest cluster. If there's only one cluster, return 1.
    """
    unique_labels, counts = np.unique(labels, return_counts=True)
    if len(counts) < 2:
        return 1.0
    return counts.max() / counts.min()


def evaluate_clustering(X, labels, true_labels=None):
    """
    Compute a set of metrics for the given clustering labels.
    Some metrics require ground truth (true_labels).
    If no true_labels is provided, ARI and MI will be omitted.
    Returns a dict of metric_name -> value.
    """
    metrics_dict = {}

    # Unsupervised metrics
    metrics_dict["Silhouette"] = silhouette_score(X, labels)
    metrics_dict["Davies-Bouldin"] = davies_bouldin_score(X, labels)
    metrics_dict["Calinski-Harabasz"] = calinski_harabasz_score(X, labels)
    metrics_dict["Diameter"] = compute_diameter(X, labels)
    metrics_dict["Split"] = compute_split(X, labels)

    # If we have ground-truth labels, we can compute supervised metrics
    if true_labels is not None:
        metrics_dict["Adjusted Rand Index"] = adjusted_rand_score(true_labels, labels)
        # Using normalized_mutual_info_score as a measure of MI
        metrics_dict["Mutual Information"] = normalized_mutual_info_score(true_labels, labels)
    return metrics_dict

In [ ]:
def kmedoids_experimentation(dataset_name, features, target, k):
    print(f"\nProcessing {dataset_name} Dataset")

    base_labels,_ = BASE_KMEDOIDS(k).fit(features)

    # Evaluate your BASE_KMEDOIDS using the evaluate_clustering function
    # base_metrics = evaluate_clustering(features, base_labels, true_labels=target.values)
    base_metrics = evaluate_clustering(features, base_labels, true_labels=target)
    print("BASE K-Means Metrics:")
    for t, v in base_metrics.items():
        print(f"  {t}: {v:.4f}")

    # Scikit-learn DBSCAN
    sklearn_labels = sklearn_kmedoids(features,k)

    # Evaluate scikit-learn's KMedoids using the evaluate_clustering function
    # sklearn_metrics = evaluate_clustering(features, sklearn_labels, true_labels=target.values)
    sklearn_metrics = evaluate_clustering(features, sklearn_labels, true_labels=target)

    print("Scikit-learn K-Medoids Metrics:")
    for t, v in sklearn_metrics.items():
        print(f"  {t}: {v:.4f}")
    return base_labels, sklearn_labels


In [ ]:
base_labels_iris, sklearn_labels_iris = kmedoids_experimentation("Iris", iris_data, iris_labels, 3)


Processing Iris Dataset
BASE K-Means Metrics:
  Silhouette: 0.5435
  Davies-Bouldin: 0.6760
  Calinski-Harabasz: 552.2292
  Diameter: 2.6089
  Split: 1.5000
  Adjusted Rand Index: 0.7028
  Mutual Information: 0.7277
Scikit-learn K-Medoids Metrics:
  Silhouette: 0.5200
  Davies-Bouldin: 0.6690
  Calinski-Harabasz: 520.4530
  Diameter: 2.5166
  Split: 1.6316
  Adjusted Rand Index: 0.7583
  Mutual Information: 0.7857


In [ ]:
ai_global_base_labels, ai_global_sklearn_labels = kmedoids_experimentation('AI Global Index', ai_data, ai_labels, 5)


Processing AI Global Index Dataset
BASE K-Means Metrics:
  Silhouette: 0.1470
  Davies-Bouldin: 1.9280
  Calinski-Harabasz: 8.3775
  Diameter: 2.7597
  Split: 7.0000
  Adjusted Rand Index: 0.0409
  Mutual Information: 0.2990
Scikit-learn K-Medoids Metrics:
  Silhouette: 0.0533
  Davies-Bouldin: 2.5113
  Calinski-Harabasz: 6.6016
  Diameter: 2.8825
  Split: 2.1111
  Adjusted Rand Index: 0.1955
  Mutual Information: 0.3684


In [ ]:
earthquake_base_labels, earthquake_sklearn_labels = kmedoids_experimentation('Earthquake', earthquake_data, alert_encoded, 4)


Processing Earthquake Dataset
BASE K-Means Metrics:
  Silhouette: 0.2183
  Davies-Bouldin: 1.6771
  Calinski-Harabasz: 127.3766
  Diameter: 13.1303
  Split: 5.8070
  Adjusted Rand Index: 0.0837
  Mutual Information: 0.1729
Scikit-learn K-Medoids Metrics:
  Silhouette: 0.2840
  Davies-Bouldin: 1.3103
  Calinski-Harabasz: 154.3139
  Diameter: 13.2360
  Split: 2.7647
  Adjusted Rand Index: -0.0003
  Mutual Information: 0.0872


/usr/local/lib/python3.11/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(


### 4.1 Predicting alert-level based on clusters and analyzing how good the prediction is.

In [ ]:
earthquake_base_labels, earthquake_sklearn_labels = kmedoids_experimentation('Earthquake', earthquake_data, alert_encoded, 4)


Processing Earthquake Dataset
BASE K-Means Metrics:
  Silhouette: 0.1221
  Davies-Bouldin: 2.3060
  Calinski-Harabasz: 82.5489
  Diameter: 13.3266
  Split: 2.5607
  Adjusted Rand Index: -0.0178
  Mutual Information: 0.0495
Scikit-learn K-Medoids Metrics:
  Silhouette: 0.2840
  Davies-Bouldin: 1.3103
  Calinski-Harabasz: 154.3139
  Diameter: 13.2360
  Split: 2.7647
  Adjusted Rand Index: -0.0003
  Mutual Information: 0.0872


/usr/local/lib/python3.11/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(


### 4.2 Reducing the number of attributes to 5 using Mutual Information Scores between feature and alert level.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

X = earthquake_data
y = alert_encoded

# Compute MI scores
mi_scores = mutual_info_classif(X, y)

# Select top 5 features
top_5_features = np.array(np.arange(X.shape[1]))[mi_scores.argsort()[-5:]]

print("Top 5 features most predictive of 'alert':", earthquake_df.columns[top_5_features])

X_selected = X[:, top_5_features]

earthquake_data_top_5 = X_selected

Top 5 features most predictive of 'alert': Index(['magnitude', 'latitude', 'longitude', 'mmi', 'sig'], dtype='object')


### 4.3 Running clustering experimentation for both base and library implementation for top-5 attribute of earthquake data

In [ ]:
base_labels, sklearn_labels = kmedoids_experimentation('Earthquake', earthquake_data, alert_encoded, 4)


Processing Earthquake Dataset
BASE K-Means Metrics:
  Silhouette: 0.1711
  Davies-Bouldin: 2.1055
  Calinski-Harabasz: 96.1199
  Diameter: 13.3830
  Split: 2.7010
  Adjusted Rand Index: -0.0107
  Mutual Information: 0.0558
Scikit-learn K-Medoids Metrics:
  Silhouette: 0.2840
  Davies-Bouldin: 1.3103
  Calinski-Harabasz: 154.3139
  Diameter: 13.2360
  Split: 2.7647
  Adjusted Rand Index: -0.0003
  Mutual Information: 0.0872


/usr/local/lib/python3.11/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 1 is empty! self.labels_[self.medoid_indices_[1]] may not be labeled with its corresponding cluster (1).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn_extra/cluster/_k_medoids.py:329: UserWarning: Cluster 3 is empty! self.labels_[self.medoid_indices_[3]] may not be labeled with its corresponding cluster (3).
  warnings.warn(
